<a href="https://colab.research.google.com/github/b028888/firts_project/blob/master/arbbitlot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4 lxml pandas -q
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [5]:
url = 'https://torgi.arbbitlot.ru/public/public-offers/lots/view/30641/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers, timeout=10)

if response.status_code == 200:
    print("? Страница успешно загружена!")
    html_content = response.text
    print(f"?? Размер HTML: {len(html_content):,} символов")

    # Сохранение полного HTML в файл
    with open('fedresurs_trade_list.html', 'w', encoding='utf-8') as f:
        f.write(html_content)
    print("?? HTML сохранен в fedresurs_trade_list.html")

else:
    print(f"? Ошибка: {response.status_code}")

? Страница успешно загружена!
?? Размер HTML: 102,436 символов
?? HTML сохранен в fedresurs_trade_list.html


In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

url = 'https://old.bankrot.fedresurs.ru/TradeList.aspx'

# Настройки с повторами и таймаутом 30 сек
session = requests.Session()
retry_strategy = Retry(
    total=3,  # 3 попытки
    backoff_factor=1,  # Задержка между повторами
    status_forcelist=[429, 500, 502, 503, 504]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# Запрос с таймаутом 30 сек
try:
    print("⏳ Загрузка Fedresurs (30 сек таймаут)...")
    response = session.get(url, headers=headers, timeout=(30, 90))  # (connect, read)

    if response.status_code == 200:
        print("✅ Успех!")
        soup = BeautifulSoup(response.text, 'lxml')
        tables = soup.find_all('table')
        print(f"📊 Найдено таблиц: {len(tables)}")

        # Все таблицы
        all_dfs = pd.read_html(response.text)
        for i, df in enumerate(all_dfs[:9], 1):
            print(f"\n📋 Таблица #{i}: {df.shape}")
            df.to_csv(f'table_{i}.csv', index=False)

    else:
        print(f"❌ Код ответа: {response.status_code}")

except requests.exceptions.Timeout:
    print("⏰ Таймаут! Сервер слишком медленный.")
except Exception as e:
    print(f"❌ Ошибка: {e}")


⏳ Загрузка Fedresurs (30 сек таймаут)...


⏰ Таймаут! Сервер слишком медленный.
